# Emissions to forcing for aerosol

Should be relatively straightforward.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl

from fair21.forcing.aerosol import radiation, cloud
from fair21.defaults import slcf_list
from fair21.defaults.short_lived_forcers import pre_industrial_emissions

In [ ]:
# grab some emissions
emissions = {}
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for specie in slcf_list:
    specie_rcmip_name = specie.replace("-", "")
    emissions[specie] = df.loc[
        (df['Scenario']=='ssp245') & (df['Variable'].str.endswith("|"+specie_rcmip_name)) & (df['Region']=='World'), '1750':
    ].interpolate(axis=1).values.squeeze()

In [ ]:
# initialise dicts for outputs. These will eventually be moved to the forward model
n_timesteps = len(emissions[specie])

In [ ]:
erf_ari = radiation.linear(
    emissions,
    pre_industrial_emissions
)

erf_aci = cloud.smith2021(
    emissions,
    pre_industrial_emissions
)

In [ ]:
len(erf_ari.keys())

In [ ]:
fig, ax = pl.subplots(2, 3, figsize=(16,9))
for icomp, component in enumerate(erf_ari):
    iy = icomp % 3
    ix = icomp // 3
    ax[ix, iy].plot(np.arange(1750.5, 2501), erf_ari[component], label='FaIR 2.1')
    ax[ix, iy].set_title(component)
ax[1,2].plot(np.arange(1750.5, 2501), erf_aci)
ax[1,2].set_title('Aerosol|Aerosol-cloud interactions')
fig.tight_layout()

In [ ]:
pl.plot(np.arange(1750.5, 2501), erf_ari['Aerosol|Aerosol-radiation interactions'] + erf_aci)